# Phụ


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import warnings
import polars as pl
warnings.filterwarnings('ignore', category=FutureWarning)

sns.set(style="whitegrid")
%matplotlib inline

train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

# Kiểm tra phân phối nhãn
print(train['sii'].value_counts(normalize=True))

sii
0.0    0.582602
1.0    0.266813
2.0    0.138158
3.0    0.012427
Name: proportion, dtype: float64


# Random Forest V3

* Sử dụng Bayesian Optimization thay vì RandomizedSearch cho HyperparameterstunningHyperparameterstunning

In [2]:
import pandas as pd
import numpy as np
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold , RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.stats import randint, uniform
import time
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Tắt cảnh báo
warnings.filterwarnings('ignore', category=FutureWarning)

# Đọc dữ liệu
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

#Sử dụng SimpleImputer cho cột 'sii' 
sii_imputer = SimpleImputer(strategy='median')
train['sii'] = sii_imputer.fit_transform(train[['sii']])

# Chuyển đổi sang integer
train['sii'] = train['sii'].astype(int)

# Bỏ các cột sii, id và cả PCIAT_PCIAT_*
X = train.drop(columns=['sii', 'id'] + [col for col in train.columns if 'PCIAT' in col])
y = train['sii']

# Xác định các loại cột
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Tạo preprocessor - Xử lý missing value
preprocessor = ColumnTransformer(
    transformers=[

        # Với dữ liệu loại số: Lấy giá trị trung bình của các row lân cận
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),

        # Với dữ liệu loại categorical: Lấy theo tần số xuất hiện nhiều nhất và encode có thành số ví dụ 0,1,...
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
        ]), categorical_features)
    ])

# Tạo pipeline với preprocessor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Định nghĩa Bayesian search space
search_spaces = {
    'classifier__n_estimators': Integer(50, 500),
    'classifier__max_depth': Categorical([None] + list(range(10, 50, 5))),
    'classifier__min_samples_split': Integer(2, 20),
    'classifier__min_samples_leaf': Integer(1, 10),
    'classifier__max_features': Categorical(['sqrt', 'log2', None]),
    'classifier__criterion': Categorical(['gini', 'entropy']),
    'classifier__bootstrap': Categorical([True, False])
}

# Thiết lập K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

start_time = time.time()

# Thực hiện Bayesian Search
bayes_search = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_spaces,
    n_iter=50,  # Số lượng cấu hình để thử
    cv=kfold,
    n_jobs=-1,
    verbose=0,
    scoring='f1_weighted'
)

bayes_search.fit(X, y)

end_time = time.time()
#In ra tổng thời gian tìm parameter phù hợp
print(f"Total training time for Random Search: {end_time - start_time:.2f} seconds")

print("\nBest parameters:", bayes_search.best_params_)
print("Best cross-validation score:", bayes_search.best_score_)

# Lấy mô hình tốt nhất từ Random Search
final_model = bayes_search.best_estimator_

# HUẤN LUYỆN LẠI TRÊN TOÀN BỘ TẬP DỮ LIỆU
final_model.fit(X, y)

# Chuẩn bị dữ liệu test
test_X = test.drop(columns=['id'] + [col for col in test.columns if 'PCIAT' in col])

# Dự đoán trên tập dữ liệu test
test_predictions = final_model.predict(test_X)

# Tạo tệp submission
submission = pd.DataFrame({
    'id': test['id'],
    'sii': test_predictions
})

submission.to_csv('submission.csv', index=False)
# submission


Total training time for Random Search: 1579.91 seconds

Best parameters: OrderedDict([('classifier__bootstrap', False), ('classifier__criterion', 'gini'), ('classifier__max_depth', 15), ('classifier__max_features', None), ('classifier__min_samples_leaf', 10), ('classifier__min_samples_split', 15), ('classifier__n_estimators', 181)])
Best cross-validation score: 0.6695477977586577
